In [ ]:
import pybullet as p
import pybullet_data
import os
import sys
import numpy as np
import time

#From original file
#robot_path = r"C:\Users\snowl\Google Drive\Documents\Research\ARCLAB Arm_\CAD\active\full_arm\arm\subarms\output\urdf\output.urdf"
#robot_path2 = r"C:\Users\snowl\Google Drive\Documents\Research\ARCLAB Arm_\CAD\active\full_arm\arm\subarms\multi.SLDASM\urdf\multi.SLDASM.urdf"
#robot_path3 = r"C:\Users\snowl\Google Drive\Documents\Research\ARCLAB Arm_\CAD\active\full_arm\arm\subarms\asdf\urdf\multi.SLDASM.urdf"
#robot_4seg = r"C:\Users\snowl\out\urdf\works.urdf"

#file paths
cwd = os.getcwd()
subfolders = ['mri','out']
urdfs = ['mri_vhacd.urdf', 'works.urdf']
model_paths = []

for folder, urdf in zip(subfolders, urdfs):
    path = os.path.join(cwd, 'models', folder, 'urdf', urdf)
    model_paths.append(path)
    
print(model_paths)
#can also use relative paths
#p.loadURDF("models/out/urdf/works.urdf")

In [ ]:
p.connect(p.GUI)#start simulation in a gui
p.setAdditionalSearchPath(pybullet_data.getDataPath())

In [ ]:
p.resetSimulation()
p.loadURDF('plane.urdf')

mri_start_pos = [-0.35,1,0.60]
mri_start_orientation = p.getQuaternionFromEuler([0,0,0])
mri = p.loadURDF(model_paths[0], mri_start_pos, mri_start_orientation, useFixedBase = 1)

robot_start_pos = [0,0,1]
#robot_start_orientation = p.getQuaternionFromEuler([0,0,0])
robot_start_orientation = p.getQuaternionFromEuler([3*np.pi/2,0,0])
robot = p.loadURDF(model_paths[1], robot_start_pos, robot_start_orientation, useFixedBase = 1, flags = p.URDF_USE_SELF_COLLISION)


p.setGravity(0, 0, -9.81)   # everything should fall down
p.setTimeStep(0.005)       # this slows everything down, but let's be accurate...
p.setRealTimeSimulation(0)  # we want to be faster than real time :)

In [ ]:
#orients to one side of search space
for i in range(1000):
    p.setJointMotorControl2(robot,0,controlMode=p.POSITION_CONTROL, targetPosition = 0)
    p.setJointMotorControl2(robot,1,controlMode=p.POSITION_CONTROL, targetPosition = np.pi)
    p.setJointMotorControl2(robot,2,controlMode=p.POSITION_CONTROL, targetPosition = np.pi)
    p.setJointMotorControl2(robot,3,controlMode=p.POSITION_CONTROL, targetPosition = np.pi)
    p.stepSimulation()
    #print(p.getContactPoints())

In [ ]:
j_i = -1
k_i = -1

for i in np.linspace(np.pi,-np.pi,360):
    j_i = j_i * -1
    for j in np.linspace(np.pi,-np.pi,360):
        k_i = k_i * -1
        j = j * j_i
        for k in np.linspace(np.pi,-np.pi,360):
            k = k * k_i
            p.setJointMotorControl2(robot,3,controlMode=p.POSITION_CONTROL, targetPosition = k)
            p.setJointMotorControl2(robot,2,controlMode=p.POSITION_CONTROL, targetPosition = j)
            p.setJointMotorControl2(robot,1,controlMode=p.POSITION_CONTROL, targetPosition = i)
            p.setJointMotorControl2(robot,0,controlMode=p.POSITION_CONTROL, targetPosition = 0)
            p.stepSimulation()
            #p.getLinkState(robot, 4,0,1)[0] #End link center of mass world coordinates
            
            #time.sleep(0.000001)
            

In [ ]:
#testing collisions
p.setTimeStep(0.01)       # this slows everything down, but let's be accurate...
p.stepSimulation()
p.getContactPoints(mri,robot)

In [ ]:
#Joint information
position,orientation = p.getBasePositionAndOrientation(robot)
print("Base orientation of robot(euler): {}".format(p.getEulerFromQuaternion(orientation)))
print("Base position of robot: {}".format(position))

num_joints = p.getNumJoints(robot) 
print("Number of joints: {}".format(num_joints))

joint_info = []
for i in range(num_joints):
    joint_info.append(p.getJointInfo(robot, i))
print(joint_info[4])
    
print("Position, velocity ,forces, torque of joint 0: {}".format(p.getJointStates(robot, [0]))) #joint position, velocity, reaction forces, and motor torque

#get world position of links
world_position, world_orientation = p.getLinkState(robot, 0)[:2]
print("world position: {}".format(world_position))
print("world orientation: {}".format(p.getEulerFromQuaternion(world_orientation)))

In [ ]:
for i in range(num_joints):
    print(joint_info[i])

In [ ]:
for _ in range(10000):
    p.stepSimulation()

In [ ]:
p.getJointState(robot, 1)

In [ ]:
p.setJointMotorControl2(robot, 1, controlMode=p.POSITION_CONTROL, targetPosition = 1)
for _ in range(10000):
    p.stepSimulation()

In [ ]:
#p.setJointMotorControlArray(
#    temp, range(num_joints), p.POSITION_CONTROL,
#    targetPositions=[1] * num_joints)
p.setJointMotorControl2(robot, 0,
     controlMode=p.POSITION_CONTROL, targetPosition = 2)

jointPositions = np.linspace(0,2, num = 100)
for i in range(100):
    p.setJointMotorControl2(bodyUniqueId = robot, jointIndex = 4, 
                            controlMode = p.POSITION_CONTROL,
                            targetPosition = 0)
    p.setJointMotorControl2(bodyUniqueId = robot, jointIndex = 1, 
                            controlMode = p.POSITION_CONTROL,
                            targetPosition = 0)
    p.setJointMotorControl2(bodyUniqueId = robot, jointIndex = 2, 
                            controlMode = p.POSITION_CONTROL,
                            targetPosition = 0)
    p.setJointMotorControl2(bodyUniqueId = robot, jointIndex = 3, 
                            controlMode = p.POSITION_CONTROL,
                            targetPosition = 0)
    p.stepSimulation()
    time.sleep(0.05)
#p.disconnect()

In [ ]:
p.disconnect()